# THE 8-PUZZLE

## STANDARD FORMULATION

Including path to previous directory in built-in variable `sys.path`:

In [1]:
import sys

sys.path.append('../')

Importing the standard formulation:

In [2]:
import numpy as np
from collections import deque
from puzzle import formulations

Class with a search problem:

In [3]:
class Problem:
    def __init__(self, initial, objective, actions, result):
        self.initial = initial
        self.objective = objective
        self.actions = actions
        self.result = result  # defines the transition model

Class with a tree node:

In [4]:
class Node:
    def __init__(self, state, cost, parent, action):
        self.state = state
        self.cost = cost
        self.parent = parent
        self.action = action

    def __str__(self):  # representing the node
        return '\n\n'.join(['  '.join(map(str, grid)) for grid in self.state[0:2]]) + \
               '  >>>  ' + str(self.cost) + '\n\n' + \
               '\n\n'.join(['  '.join(map(str, grid)) for grid in self.state[2:]])

    @classmethod
    def child(cls, problem, parent, action):
        state = problem.result(parent.state, action)
    
        return cls(state, parent.cost + 1, parent, action)

    @property
    def solution(self):
        node = self;
        solution = []

        while node:
            solution.append(node)
            node = node.parent

        solution.reverse()

        return solution

Creating new instances of the problem:

In [5]:
grid1 = np.array([[3, 2, 7],
                  [4, 8, 1],
                  [6, 5, 0]])
grid2 = np.array([[3, 2, 7],
                  [4, 8, 0],
                  [6, 5, 1]])
grid3 = np.array([[3, 2, 0],
                  [4, 8, 7],
                  [6, 5, 1]])

puzzle_12_steps = Problem(Node(grid1, 0, None, None), formulations.won,
                          formulations.available_moves, formulations.move_grid)

puzzle_13_steps = Problem(Node(grid2, 0, None, None), formulations.won,
                          formulations.available_moves, formulations.move_grid)

puzzle_14_steps = Problem(Node(grid3, 0, None, None), formulations.won,
                          formulations.available_moves, formulations.move_grid)

## BREADTH-FIRST SEARCH (EFFICIENT FOR PATHS WITH UP TO 12 STEPS)

Constants for current search status:

In [6]:
SEARCH_NOT_STARTED = 0
SEARCH_STARTED = 1
SEARCH_FAIL = 2
SEARCH_SUCCESS = 3

Class that will implement breadth-first search:

In [7]:
class BFS:
    def __init__(self, problem):
        self.problem   = problem
        self.explored  = deque([])
        self.frontier  = deque([])
        self.situation = SEARCH_NOT_STARTED
        self.solution  = None

    def step_search(self):
        if self.situation == SEARCH_NOT_STARTED:
            self.frontier.append(self.problem.initial)
            self.situation = SEARCH_STARTED
        elif self.situation == SEARCH_FAIL:
            print("Search process failed!")
            return
        elif self.situation == SEARCH_SUCCESS:
            print("Solution already found!")
            return

        # Empty border ends the search
        if not self.frontier:
            self.situation = SEARCH_FAIL
            return

        # Performing the search step
        node = self.frontier.popleft()
        
        self.explored.append(node.state)

        for action in self.problem.actions(node.state):
            child = Node.child(self.problem, node, action)
            
            if child not in self.frontier and not self.explored_node(child.state):
                if self.problem.objective(child.state):
                    self.solution = child.solution
                    self.situation = SEARCH_SUCCESS
                    return
                
                self.frontier.append(child)

    def search(self):
        # Loop that performs breadth-first search
        while self.situation == SEARCH_STARTED or self.situation == SEARCH_NOT_STARTED:
            self.step_search()

        if self.situation == SEARCH_FAIL:
            print('Search process failed!')
        else:
            print('Solution found!')
        
    def explored_node(self, state):
        for state_explored in self.explored:
            if np.array_equal(state, state_explored):
                return True
            
        return False

    @property
    def show_solution(self):
        if self.situation == SEARCH_SUCCESS:
            return '\n\n'.join([node.__str__() for node in self.solution]) + \
                  f'\n\nCost: {self.solution[-1].cost}'
    
        return 'Solution still not found!'

    @property
    def show_frontier(self):
        return '#'*15 + '\n' + \
               '\n'.join([node.__str__() for node in self.frontier]) + \
               '\n' + '#'*15

Testing the efficiency of the breadth-first search:

In [8]:
%%time

bfs = BFS(puzzle_12_steps)
bfs.search()

Solution found!
CPU times: total: 3.27 s
Wall time: 3.31 s


In [9]:
%%time

bfs = BFS(puzzle_13_steps)
bfs.search()

Solution found!
CPU times: total: 12.7 s
Wall time: 12.8 s


In [10]:
%%time

bfs = BFS(puzzle_14_steps)
bfs.search()

Solution found!
CPU times: total: 27.8 s
Wall time: 28.5 s


Breadth-first search via function:

In [11]:
def bfs(problem):
    node = problem.initial
    
    if problem.objective(node.state):
        return node.solution

    explored = deque([])
    frontier = deque([node])
    
    while True:
        if not frontier:
            print("Search process failed!")
            return None

        # Performing the search step
        node = frontier.popleft()
        
        explored.append(node.state)
        
        for action in problem.actions(node.state):
            child = Node.child(problem, node, action)
            
            visited = False
            for state_explored in explored:
                if np.array_equal(child.state, state_explored):
                    visited = True
                    break
            
            if child not in frontier and not visited:
                if problem.objective(child.state):
                    print("Solution found!")
                    return child.solution
      
                frontier.append(child)

def show_solution(solution):
    print('\n\n'.join([node.__str__() for node in solution]) + \
         f'\n\nCost: {solution[-1].cost}')

Checking the efficiency:

In [12]:
%%time

_ = bfs(puzzle_12_steps)

Solution found!
CPU times: total: 3.38 s
Wall time: 3.41 s


In [13]:
%%time

_ = bfs(puzzle_13_steps)

Solution found!
CPU times: total: 12.9 s
Wall time: 13.1 s


In [14]:
%%time

_ = bfs(puzzle_14_steps)

Solution found!
CPU times: total: 27.6 s
Wall time: 27.8 s


## BACKTRACKING SEARCH (EFFICIENT FOR PATHS WITH UP TO 12 STEPS)

In [15]:
reverse_state = {'l': 'r',
                 'r': 'l',
                 'd': 'u',
                 'u': 'd'}

def backtracking(problem, limit):
    solution = np.array([None for i in range(limit+1)])
    
    size = 0
    solution[size] = (problem.initial.state,
                      problem.actions(problem.initial.state))
    
    while not problem.objective(solution[size][0]):        
        if size == limit:
            size -= 1
            
            while not solution[size][1]:
                size -= 1
                
                if size == -1:
                    return None

        state = problem.result(solution[size][0], solution[size][1].pop())

        size += 1
        solution[size] = (state, problem.actions(state))
    
    return solution

Testing backtracking search:

In [16]:
%%time

solution = backtracking(puzzle_12_steps, 12)

len(solution)

CPU times: total: 1.22 s
Wall time: 1.24 s


13

In [17]:
%%time

solution = backtracking(puzzle_13_steps, 13)

len(solution)

CPU times: total: 1.19 s
Wall time: 1.26 s


14

In [18]:
%%time

solution = backtracking(puzzle_14_steps, 14)

len(solution)

CPU times: total: 1.22 s
Wall time: 1.25 s


15